In [ ]:
''' A-Priori Algorithm implemented from the scratch'''
import os
import sys
import re
import findspark
import math
findspark.init()
from pyspark import SparkContext,SparkConf
conf=SparkConf()
sc = SparkContext(conf=conf )
'''
array=[0 for i in range(10000)]
l=[2]
def count(x,l):
    s=0
    for x in range(1000):
        #if x not in l:
        s+=l[0]
        #else:
         #   continue
    return s
    
res=sc.parallelize(array).map(lambda x:count(x,l))
print(res.take(3))
'''
text = sc.textFile("browsing.txt")
text=text.map(lambda x:x.strip().split(' '))
length=text.count()
count=text.flatMap(lambda x:x).map(lambda x:(x,1))
freq=count.reduceByKey(lambda x,y:x+y).filter(lambda x:x[1]>=100)
print("Number of frequent single itemsets -->",freq.count())
countIDs=freq.collectAsMap()#count.countByKey()
def create_pairs(x):
    temp=[]
    for i in range(len(x)):
        for j in range(i+1,len(x)):
            if x[i] in countIDs.keys() and x[j] in countIDs.keys():
                key=tuple(sorted((x[i],x[j])))
                temp.append(key)
    return temp
text=text.map(lambda x:create_pairs(x))

count=text.flatMap(lambda x:x).map(lambda x:(x,1))
freq=count.reduceByKey(lambda x,y:x+y).filter(lambda x:x[1]>=100)
print("Number of frequent pair itemsets -->",freq.count())
countIDs1=freq.collectAsMap()#count.countByKey()

def compute_pair_confidence(x):
    temp=[]
    for i in range(len(x[0])):
        temp.append(x[1]/countIDs[x[0][i]])
    idx=temp.index(max(temp))
    ret=(x[0][idx],)
    for i in range(len(x[0])):
        if i!=idx:
            ret+=('--->',x[0][i])
    confidence=max(temp)
    return ret+(confidence,math.fabs(confidence-countIDs[x[0][idx]]/length))
#Compute the Interesting Associations Rules for pair itemsets

Association_Rule_1=freq.map(lambda x:compute_pair_confidence(x))
print("Association for two pair frequent Itemsets Rule with Conf>0.95)",sorted(Association_Rule_1.filter(lambda x:x[-1]>0.95).take(8)))
def create_triplets(l):
    temp=[]
    for i in range(len(l)):
        for j in range(i+1,len(l)):
            for z in l[j]:
                if z not in l[i]:
                    t=tuple(sorted(l[i]+(z,)))
                    if t not in temp and l[i] in countIDs1.keys():
                        temp.append(t)
    i=0
    while(i<len(temp)):
        flag=True
        for j in range(0,len(temp[i])):
            for k in range(j+1,len(temp[i])):
                if tuple(sorted((temp[i][j],temp[i][k]))) not in l:
                    del temp[i]
                    flag=False
                    break
            if flag==False:
                i=i-1
                break
        i+=1
    return temp

text=text.map(lambda x:create_triplets(x))

print("Triplets --->",text.take(3))
count=text.flatMap(lambda x:x).map(lambda x:(x,1))
freq=count.reduceByKey(lambda x,y:x+y).filter(lambda x:x[1]>=100)
print("Number of frequent triple itemsets-->",freq.count())
#countIDs2=freq.collectAsMap()#count.countByKey()
print(freq.take(3))
def compute_triplet_confidence(x):
    temp=[]
    for i in range(len(x[0])-1):
        temp.append(((x[0][i],),countIDs[x[0][i]]))
        for j in range(i+1,len(x[0])):
                    key=tuple(sorted((x[0][i],x[0][j])))
                    temp.append((key,countIDs1[key]))
    I,support=min(temp,key=lambda x:x[1])
    j=()
    for i in x[0]:
        if i not in I:
            j+=(i,)
    confidence=x[1]/support
    if len(j)==1:
        probability=countIDs[j[0]]/length
    else:
        probability=countIDs1[j]/length
    return I+('--->',j,confidence,math.fabs(confidence-probability))              
        
Association_Rule_2=freq.map(lambda x:compute_triplet_confidence(x))
print("Association for three pair frequent Itemsets Rule with Conf>0.55)",sorted(Association_Rule_2.filter(lambda x:x[-2]>0.55).take(20)))
print(Association_Rule_2.take(2))
sc.stop()


Number of frequent single itemsets --> 647
Number of frequent pair itemsets --> 1334
Association for two pair frequent Itemsets Rule with Conf>0.95) [('DAI43868', '--->', 'SNA82528', 0.972972972972973, 0.963455594110557), ('DAI88079', '--->', 'FRO40251', 0.9867256637168141, 0.9721923689674492), ('DAI93865', '--->', 'FRO40251', 1.0, 0.9933121121507347), ('ELE12951', '--->', 'FRO40251', 0.9905660377358491, 0.9871577871972812), ('FRO92469', '--->', 'FRO40251', 0.983510011778563, 0.9562118541630522), ('GRO38636', '--->', 'FRO40251', 0.9906542056074766, 0.9872138017619411), ('GRO85051', '--->', 'FRO40251', 0.999176276771005, 0.9601421621123123)]
Triplets ---> [[('ELE17451', 'GRO99222', 'SNA90258')], [('ELE26917', 'ELE52966', 'GRO99222'), ('ELE26917', 'GRO99222', 'SNA30755'), ('ELE17451', 'ELE26917', 'GRO99222'), ('ELE26917', 'GRO99222', 'SNA80192'), ('ELE26917', 'GRO12298', 'GRO99222'), ('ELE26917', 'GRO99222', 'SNA11465'), ('ELE17451', 'GRO99222', 'SNA80192'), ('ELE17451', 'GRO12298', 'GRO